In [1]:
import json
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import random
import json

import plotly.graph_objects as go


In C:\Users\zande\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\zande\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\zande\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\zande\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\zande\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

## Prepare station data

In [2]:
# Load station data
df = pd.read_csv('inputs/ems_stations.csv')
print(df.head())
print(len(df))


  station_name  longitude   latitude
0    Demand 01 -97.721725  30.259487
1  Demand 02   -97.860269  30.156364
2  Demand 03   -97.750395  30.246625
3   Demand 04  -97.661592  30.323391
4    Demand 05 -97.748300  30.265327
44


In [3]:
with open('inputs/austin_amb_deployment.json') as f:
    austin_data = json.load(f)

print(austin_data.keys())
print(austin_data['Stochastic']['40'])

dict_keys(['Stochastic', 'MEXCLP', 'MALP'])
[2, 2, 2, 1, 0, 0, 1, 2, 2, 0, 0, 0, 0, 1, 1, 1, 4, 3, 1, 0, 0, 2, 0, 1, 1, 2, 0, 0, 1, 1, 1, 1, 1, 2, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1]


In [4]:
# Append data to dataframe
df['amb_deployment'] = austin_data['Stochastic']['40']
print(df.head())

df['text'] = df['station_name'] + '<br>Ambulances Deployed = ' + (df['amb_deployment']).astype(str)


  station_name  longitude   latitude  amb_deployment
0    Demand 01 -97.721725  30.259487               2
1  Demand 02   -97.860269  30.156364               2
2  Demand 03   -97.750395  30.246625               2
3   Demand 04  -97.661592  30.323391               1
4    Demand 05 -97.748300  30.265327               0


## Get boundary of Travis County

In [5]:
import requests
response = requests.get("https://geo.traviscountytx.gov/arcgis/rest/services/TNR/TravisCountyBoundary/MapServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json")
boundary = response.json()['features'][0]['geometry']['paths'][0]
print(boundary[:10])

[[-98.02690146055397, 30.25180859020622], [-98.06617710862004, 30.279938744849147], [-98.0898169587, 30.297347648499557], [-98.17313525500629, 30.356230266489355], [-98.15083794860637, 30.388880052495747], [-98.12549451239816, 30.426201225529997], [-98.09774995578586, 30.467628390481256], [-98.09787919554017, 30.498081677988523], [-98.10128859858929, 30.497395666001097], [-98.10384256600935, 30.496481774745988]]


## Create bubble map

In [7]:
import plotly.graph_objects as go

#mapbox_access_token = open(".mapbox_token").read()
mapbox_access_token = 'pk.eyJ1IjoiZmFybWVyZ3V5Y2YiLCJhIjoiY2tqMjNsMXdwMzRmajJ1cnhqdzQwN2JiYyJ9.9kWHdrox4GBO4Xow1z4wOw'
scale = 2
base = 4

fig = go.Figure(go.Scattermapbox(
        lat = df['latitude'],
        lon = df['longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size= (df['amb_deployment'] * scale) + base
        ),
        text=df['text'],
    ))

fig.update_layout(
    hovermode='closest',
    mapbox={
        'accesstoken':mapbox_access_token,
        'bearing':0,
        'center':go.layout.mapbox.Center(
            lat=30.2672,
            lon=-97.7431
        ),
        'pitch':0,
        'zoom':8,
        
        'layers': [{
            'source': {
                'type': "FeatureCollection",
                'features': [{
                    'type': "Feature",
                    'geometry': {
                        'type': "MultiPolygon",
                        'coordinates': [[boundary]]
                    }
                }]
            },
            'type': "line", 'below': "traces", 'color': "lightskyblue"}]}
)

fig.show()